# Observing conditions check

> Check that the observing conditions meet requirements.

In [ ]:
# |default_exp diagnostics.obs_cond_check

In [ ]:
# |export

import xarray as xr

from qagmire.data import (
    get_lr_l1_single_files,
    read_primary_header,
)
from qagmire.quality_assurance import Diagnostics
from qagmire.utilities import add_expid, parse_obstemp

To write checks of the data, we create a subclass of `Diagnostics` and implement the `tests` method.

In [ ]:
# |export


class ObsCondCheck(Diagnostics):
    """Observing conditions check.

    A reproduction of the weaveio [obs_cond_check](https://github.com/bamford/QAG/blob/master/diagnostics/obs_cond_checks.py).

    This tests for the following cases:

    * Is the sky brighter than the requirement?
    * Is the seeing worse than the requirement?

    and some supplementary tests, if `by_exposure=True`:

    * Are there other than two runs with the same EXPID?
    * Do runs with the same EXPID have different sky brightness?
    * Do runs with the same EXPID have different seeing?

    or if `by_exposure=False`:
    * Are there other than six runs with the same OBID?
    """

    def __init__(
        self,
        sky_tolerance: float = 0.0,  # the tolerance in the sky brightness in magnitudes
        seeing_tolerance: float = 0.0,  # the tolerance in the seeing in arcsec
        by_exposure=True,  # should the checks be performed per exposure (the default), or per OB
    ):
        self.sky_tolerance = sky_tolerance
        self.seeing_tolerance = seeing_tolerance
        self.by_exposure = by_exposure
        if by_exposure:
            self._get_and_check = self._get_and_check_by_exp
        else:
            self._get_and_check = self._get_and_check_by_ob
        super().__init__()

    @staticmethod
    def _restore_coords(coords, da):
        return [d.assign_coords(coords) for d in da]

    @classmethod
    def _get_and_check_by_exp(cls, col):
        col = col.reset_coords("CAMERA", drop=True)
        coords = (
            col.swap_dims(filename="EXPID")
            .coords.to_dataset()
            .reset_coords()
            .groupby("EXPID")
            .first()
        )
        by_exp = col.groupby("EXPID")
        count, first, last = cls._restore_coords(
            coords, (by_exp.count(), by_exp.first(), by_exp.last())
        )
        expected_runs = count == 2
        runs_match = first == last
        return first, expected_runs, runs_match

    @staticmethod
    def _get_and_check_by_ob(col):
        by_ob = col.groupby("OBID")
        with xr.set_options(use_flox=False):
            # flox does not work with .count and string arrays
            count, first = (by_ob.count(), by_ob.first())
        expected_runs = count == 6
        runs_match = (first != col).any(axis=-1)
        return first, expected_runs, runs_match

    def tests(
        self,
        **kwargs,
    ):
        files = get_lr_l1_single_files(**kwargs)
        hdr = read_primary_header(files)
        if self.by_exposure:
            hdr = add_expid(hdr)

        obstemp, count_runs, obstemp_runs_match = self._get_and_check(hdr["OBSTEMP"])
        obs = parse_obstemp(obstemp)

        self.data = (hdr, obs)

        sky, _, sky_runs_match = self._get_and_check(hdr["SKYBRTEL"])
        sky_fail = sky < obs["sky_brightness"] - self.sky_tolerance
        seeing, _, seeing_runs_match = self._get_and_check(hdr["SEEINGB"])
        seeing_fail = seeing > obs["seeing"] + self.seeing_tolerance

        tests = [
            {
                "name": "sky_too_bright",
                "description": "Is the sky brighter than the requirement?",
                "test": sky_fail,
            },
            {
                "name": "seeing_too_poor",
                "description": "Is the seeing worse than the requirement?",
                "test": seeing_fail,
            },
        ]
        if self.by_exposure:
            tests.extend(
                [
                    {
                        "name": "wrong_run_count",
                        "description": "Are there other than two runs with the same EXPID?",
                        "test": ~count_runs,
                    },
                    {
                        "name": "unmatched_runs_sky",
                        "description": "Do runs with the same EXPID have different sky brightness?",
                        "test": ~sky_runs_match,
                    },
                    {
                        "name": "unmatched_runs_seeing",
                        "description": "Do runs with the same EXPID have different seeing?",
                        "test": ~seeing_runs_match,
                    },
                ]
            )
        else:
            tests.extend(
                [
                    {
                        "name": "wrong_run_count",
                        "description": "Are there other than six runs with the same OBID?",
                        "test": ~count_runs,
                    },
                ]
            )

        return tests

## Demonstration tests

In [ ]:
tests = ObsCondCheck()
tests.run(date="201*")

Reading files:   0%|                                                                                                               | 0/126 [00:00<?, ?it/s]

Reading files:   1%|▊                                                                                                      | 1/126 [00:00<00:40,  3.08it/s]

Reading files:   2%|█▋                                                                                                     | 2/126 [00:00<00:33,  3.71it/s]

Reading files:  15%|███████████████▍                                                                                      | 19/126 [00:00<00:02, 43.05it/s]

Reading files:  25%|█████████████████████████▉                                                                            | 32/126 [00:00<00:01, 64.95it/s]

...

Reading files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 126/126 [00:01<00:00, 77.20it/s]


Creating Dataset... 

took 2.01 s. Size is 0.800 Mb


Tests took 8.05 s to prepare (including reading data).


Tests took 1.00 s to perform.
sky_too_bright:
    Is the sky brighter than the requirement?
seeing_too_poor:
    Is the seeing worse than the requirement?
wrong_run_count:
    Are there other than two runs with the same EXPID?
unmatched_runs_sky:
    Do runs with the same EXPID have different sky brightness?
unmatched_runs_seeing:
    Do runs with the same EXPID have different seeing?


In [ ]:
tests.summary_per_test()

5 varieties of test and 63 tested elements per variety, for total of 315 tests.
6 tests failed (1.90%) and 309 tests passed (98.10%).


,total fails
test,
sky_too_bright,6
seeing_too_poor,0
unmatched_runs_seeing,0
unmatched_runs_sky,0
wrong_run_count,0


In [ ]:
tests.summary()

5 varieties of test and 63 tested elements per variety, for total of 315 tests.
6 tests failed (1.90%) and 309 tests passed (98.10%).


,,,,,,failed,total fails
,,,,,test,sky_too_bright,
EXPID,filename,RUN,OBID,MJD,NIGHT,,
321700000001,single_1002310,1002310,3217,57641.001725,20160909,True,1
321700000002,single_1002312,1002312,3217,57641.014919,20160909,True,1
321700000003,single_1002313,1002313,3217,57641.028113,20160909,True,1
343400000001,single_1002250,1002250,3434,57639.999560,20160908,True,1
343400000002,single_1002251,1002251,3434,57640.012755,20160908,True,1
343400000003,single_1002253,1002253,3434,57640.025949,20160908,True,1


In [ ]:
tests.full_summary()

5 varieties of test and 63 tested elements per variety, for total of 315 tests.
6 tests failed (1.90%) and 309 tests passed (98.10%).


failed  \
test                                                           seeing_too_poor   
EXPID        filename       RUN     OBID MJD          NIGHT                      
313300000001 single_1002226 1002226 3133 57639.909236 20160908           False   
313300000002 single_1002227 1002227 3133 57639.922431 20160908           False   
313300000003 single_1002229 1002229 3133 57639.935625 20160908           False   
317000000001 single_1002286 1002286 3170 57640.911400 20160909           False   
317000000002 single_1002287 1002287 3170 57640.924595 20160909           False   
...                                                                        ...   
440700000002 single_1004123 1004123 4407 58027.050104 20170930           False   
440700000003 single_1004126 1004126 4407 58027.063299 20170930           False   
446400000001 single_1004098 1004098 4464 58026.823542 20170930           False   
446400000002 single_1004099 1004099 4464 58026.836736 20170930           False   
446400000003 single_1004101 1004101 4464 58026.849931 20170930           False   

                                                                               \
test                                                           sky_too_bright   
EXPID        filename       RUN     OBID MJD          NIGHT                     
313300000001 single_1002226 1002226 3133 57639.909236 20160908          False   
313300000002 single_1002227 1002227 3133 57639.922431 20160908          False   
313300000003 single_1002229 1002229 3133 57639.935625 20160908          False   
317000000001 single_1002286 1002286 3170 57640.911400 20160909          False   
317000000002 single_1002287 1002287 3170 57640.924595 20160909          False   
...                                                                       ...   
440700000002 single_1004123 1004123 4407 58027.050104 20170930          False   
440700000003 single_1004126 1004126 4407 58027.063299 20170930          False   
446400000001 single_1004098 1004098 4464 58026.823542 20170930          False   
446400000002 single_1004099 1004099 4464 58026.836736 20170930          False   
446400000003 single_1004101 1004101 4464 58026.849931 20170930          False   

                                                                                      \
test                                                           unmatched_runs_seeing   
EXPID        filename       RUN     OBID MJD          NIGHT                            
313300000001 single_1002226 1002226 3133 57639.909236 20160908                 False   
313300000002 single_1002227 1002227 3133 57639.922431 20160908                 False   
313300000003 single_1002229 1002229 3133 57639.935625 20160908                 False   
317000000001 single_1002286 1002286 3170 57640.911400 20160909                 False   
317000000002 single_1002287 1002287 3170 57640.924595 20160909                 False   
...                                                                              ...   
440700000002 single_1004123 1004123 4407 58027.050104 20170930                 False   
440700000003 single_1004126 1004126 4407 58027.063299 20170930                 False   
446400000001 single_1004098 1004098 4464 58026.823542 20170930                 False   
446400000002 single_1004099 1004099 4464 58026.836736 20170930                 False   
446400000003 single_1004101 1004101 4464 58026.849931 20170930                 False   

                                                                                   \
test                                                           unmatched_runs_sky   
EXPID        filename       RUN     OBID MJD          NIGHT                         
313300000001 single_1002226 1002226 3133 57639.909236 20160908              False   
313300000002 single_1002227 1002227 3133 57639.922431 20160908              False   
313300000003 single_1002229 1002229 3133 57639.935625 20160908              False   
317000000001 single_1002286 1002286 3170 5764

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()